In [1]:
from pyspark import SparkContext,SparkConf
from pyspark.sql import SparkSession
from boto import s3

def get_s3_tempdata(dataName):
    prefix = "uidcreattime-csv/"
    total_size = 0
    REGION = "cn-north-1"
    conn = s3.connect_to_region(REGION)
    bucket = conn.lookup('prodlog')
    ret = []
    if bucket:
        for k in bucket.list(prefix=prefix):
            if k.size <= 0:
                continue
            logsp = k.name.split('/')
            a = logsp[2]
            csvName = a.split('.')[0]
            if csvName == dataName:
                total_size += k.size
                ret.append('s3://prodlog/' + k.name)
                print('s3://prodlog/' + k.name, ''.join(a))
    print('total:%d' % (total_size / 1024.0 / 1024.0 / 1024.0))
    return ret


try:
    sc.stop()
except:
    pass
spark = SparkSession \
    .builder.enableHiveSupport() \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [2]:
df = spark.read.json("/Users/tq/bigdatatest/pay/20161110_payLog.json")

In [3]:
print type(df)

<class 'pyspark.sql.dataframe.DataFrame'>


In [4]:
df1 = spark.read.json("/Users/tq/bigdatatest/pay/20161110_payLog.json")

In [9]:
final_uid_Result = df.union(df1)

In [11]:
final_uid_Result.toPandas()

,LogTime,acid,ch,corpLvl,date,gid,payNum,payTime,sid,uid,vipLvl
0,1478749789,202:3001:8fba9a9a-6f6e-44f6-991e-0b074b340ce2,130134012000,2,2016-11-10,202,600,20161110034928,3001,8fba9a9a-6f6e-44f6-991e-0b074b340ce2,0
1,1478750405,202:3001:749741d8-2a22-42cb-91ab-fb297bcf4016,130134012700,10,2016-11-10,202,600,20161110035929,3001,749741d8-2a22-42cb-91ab-fb297bcf4016,0
2,1478750595,202:3001:4e4d5680-64d5-4b63-b1dd-b4706cb4d34d,130134012700,9,2016-11-10,202,9800,20161110040242,3001,4e4d5680-64d5-4b63-b1dd-b4706cb4d34d,0
3,1478749789,202:3001:8fba9a9a-6f6e-44f6-991e-0b074b340ce2,130134012000,2,2016-11-10,202,600,20161110034928,3001,8fba9a9a-6f6e-44f6-991e-0b074b340ce2,0
4,1478750405,202:3001:749741d8-2a22-42cb-91ab-fb297bcf4016,130134012700,10,2016-11-10,202,600,20161110035929,3001,749741d8-2a22-42cb-91ab-fb297bcf4016,0
5,1478750595,202:3001:4e4d5680-64d5-4b63-b1dd-b4706cb4d34d,130134012700,9,2016-11-10,202,9800,20161110040242,3001,4e4d5680-64d5-4b63-b1dd-b4706cb4d34d,0


In [35]:
final_uid_Result_fream.show()

+---+--------------------+----------+----------+----+---+------------+--------------------+
|_c0|                 uid|        lt|      date| sid|gid|          ch|             machine|
+---+--------------------+----------+----------+----+---+------------+--------------------+
|  0|0d95ac88-18bb-4ec...|1479186521|2016-11-15|4004|203|130134000144|HUAWEI HUAWEI RIO...|
|  1|10df7f3f-e417-448...|1479221949|2016-11-15|4004|203|130134000144| Xiaomi Redmi Note 4|
|  2|15b61d33-a9d6-4b4...|1479215554|2016-11-15|4004|203|130134000144|    samsung GT-I9158|
|  3|16c823c5-dffd-4ce...|1479181880|2016-11-15|4003|203|130134000144|        vivo vivo X7|
|  4|1ebdf1b9-a311-4a4...|1479234298|2016-11-16|4004|203|130134000144|        vivo vivo V3|
|  5|216353a4-7b29-43f...|1479217003|2016-11-15|4004|203|130134000144|            OPPO A51|
|  6|2637d0d8-f85c-442...|1479215239|2016-11-15|4002|203|130134000144|          Meizu M1 E|
|  7|27847852-54f6-46a...|1479204290|2016-11-15|4001|203|130134000144|     samsu

In [37]:
ss = spark.read.json("/Users/tq/Downloads/logics_shard1009.07.03.2017.log")

In [39]:
ss.createOrReplaceTempView('RawsTable')
preparetable = 'RawsTable'

result1 = spark.sql(u'''
SELECT
    *
FROM %s
WHERE accountid is not null
AND type_name!="CorpLevelChg"
''' % preparetable).cache().createOrReplaceTempView('jsonsTable')

In [40]:
# 获取当日log里的最早的一次登录
spark.sql(u"""
SELECT
    t.uid AS uid,
    SUBSTRING(t.lt,0,10) AS lt,
    CAST(jsonsTable.utc8 as DATE) AS date,
    jsonsTable.sid AS sid,
    jsonsTable.gid AS gid,
    jsonsTable.channel AS ch,
    jsonsTable.info.MachineType AS machine
FROM (
    SELECT
        userid AS uid,
        min(logtime) AS lt
    FROM jsonsTable
    WHERE type_name="Login"
    GROUP BY userid
) t
LEFT JOIN jsonsTable ON jsonsTable.userid = t.uid AND jsonsTable.logtime = t.lt
""").cache().createOrReplaceTempView('uid_ch')

In [41]:
acidlog_df = spark.sql(u"""
SELECT *
FROM uid_ch
""")

In [42]:
def min_time(x, y):
    if int(x.lt) > int(y.lt):
        return y
    else:
        return x

In [44]:
df_uid = spark.read.csv("/Users/tq/Downloads/uid_creatTime.csv",header = "true")

In [55]:
aaa = df_uid.toPandas()

In [56]:
del aaa['_c0']

In [60]:
fff = spark.createDataFrame(aaa)

In [68]:
ccc = acidlog_df.toPandas()

In [62]:
final_uid_Result = fff.rdd.union(acidlog_df.rdd).map(lambda x: (x.uid, x)).reduceByKey(min_time).values()

In [63]:
print type(final_uid_Result)

<class 'pyspark.rdd.PipelinedRDD'>


In [64]:
final_uid_Result_fream = spark.createDataFrame(final_uid_Result)

In [65]:
print type(final_uid_Result_fream)

<class 'pyspark.sql.dataframe.DataFrame'>


In [66]:
final_uid_Result_fream.toPandas()

,uid,lt,date,sid,gid,ch,machine
0,5dd3b169-33f7-4729-b1c7-08a1b50a69f7,1478760781,2016-11-10,3002,202,130134015200,Meizu m3 note
1,86589ad3-1e17-4095-8c93-ee60c24e80a0,1478753491,2016-11-10,3001,202,130134012700,OPPO OPPO R9m
2,587afa52-9220-4931-8457-3b11a0263c01,1478765969,2016-11-10,1003,200,130134001300,OPPO A31
3,fe5e4b30-8384-4d80-96dd-72544fa8f887,1478785594,2016-11-10,3002,202,130134012000,HUAWEI ATH-AL00
4,5a6b30ef-47f9-4932-be6a-e8e039a62788,1478761062,2016-11-10,3001,202,130134000137,LENOVO Lenovo K50-t3s
5,1b3a0f89-dc76-436b-a3c2-6d61e9ce1884,1478751636,2016-11-10,1001,200,130134312108.0,nubia NX511J_V3
6,058e3af8-0554-49a6-968b-e58b811dff36,1478770319,2016-11-10,1003,200,130134001500,Xiaomi MI 4LTE
7,19f4aa11-105a-4013-a9b7-c691e1936469,1478754969,2016-11-10,1001,200,130134005400,Xiaomi MI 4LTE
8,8389670a-5047-4126-b645-06173b85106e,1478770223,2016-11-10,3001,202,130134012000,HUAWEI HUAWEI MLA-AL10
9,656bf918-b23c-4859-86f8-2f380be9caae,1478763702,2016-11-10,3002,202,130134012000,HUAWEI PLK-TL01H


In [76]:
qqq = final_uid_Result_fream.toPandas()

In [67]:
import pandas as pd

In [77]:
# 合并当天支付log
frames = [aaa, qqq]
bbbb = pd.concat(frames)
print type(bbbb)

<class 'pandas.core.frame.DataFrame'>


In [78]:
hhh = spark.createDataFrame(bbbb)

In [79]:
print type(hhh)

<class 'pyspark.sql.dataframe.DataFrame'>


In [80]:
hhh.createOrReplaceTempView('payLog')